In [ ]:
pip install asana

In [ ]:
import asana
from asana.rest import ApiException
from pprint import pprint
import requests
import pandas as pd
from pandas import json_normalize
import json

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
# Configure OAuth2 access token for authorization
configuration = asana.Configuration()
configuration.access_token = 'xxxxxx' #troque pelo seu acesso
api_client = asana.ApiClient(configuration)

api_instance = asana.TasksApi(api_client)

# ID do projeto (substitua pelo ID do projeto desejado)
project_gid = 'xxxxxx' #id projeto Mesquita

In [ ]:
opts = {
    'opt_fields': ",".join([
    "gid",
    "created_at",
    "completed_at",
    "modified_at",
    "name",
    "memberships.section.name",
    "assignee.name",
    "assignee.email",
    "start_on",
    "due_on",
    "tags.name",
    "notes",
    "projects.name",
    ])
}

def get_all_tasks_and_subtasks(task, parent_task_name=None):
    task_data.append({
        'section_name': task.get('memberships', [{}])[0].get('section', {}).get('name', None) if task.get('memberships', []) else None,
        'task_name': parent_task_name if parent_task_name else task.get('name', None),
        'subtask_name': task.get('name', None),
        'assignee': task.get('assignee', {}).get('name', None) if task.get('assignee', None) else None,
        'subtask_notes': task.get('notes', None),
        'due_on': pd.to_datetime(task.get('due_on', None)).strftime("%d/%m/%Y") if task.get('due_on', None) else None,
        'tag_names': [tag.get('name', None) for tag in task.get('tags', [])] if task.get('tags', None) else None
    })

    subtask_response = api_instance.get_subtasks_for_task(task.get('gid'), opts=opts)
    subtasks = list(subtask_response)  # Transformando o gerador em uma lista

    for subtask in subtasks:
        get_all_tasks_and_subtasks(subtask, parent_task_name=parent_task_name if parent_task_name else task.get('name', None))




try:
    project_tasks_response = api_instance.get_tasks_for_project(project_gid, opts=opts)
    project_tasks = list(project_tasks_response)  # Transformando o gerador em uma lista
except ApiException as e:
    if e.status == 401:
        print("Erro de autorização. Verifique seu token de acesso e permissões.")
    else:
        print(f"Exception when calling TasksApi->get_tasks_for_project: {e}")
    exit()

# Lista para armazenar os dados de tarefas e subtarefas
task_data = []

# Iterar sobre as tarefas do projeto
for task in project_tasks:
    # Certifique-se de que a tarefa é um dicionário antes de processá-la
    if isinstance(task, dict):
        get_all_tasks_and_subtasks(task)
    else:
        print(f"Objeto inesperado encontrado: {task}")

# Crie um DataFrame a partir dos dados coletados
df = pd.DataFrame(task_data)

# Dicionário de mapeamento para tradução de nomes de colunas
nome_coluna_map = {
    'section_name': 'Seção',
    'task_name': 'Nome da Tarefa',
    'subtask_name': 'Nome da Subtarefa',
    'assignee': 'Responsável',
    'subtask_notes': 'Notas',
    'due_on': 'Concluída em',
    'tag_names': 'Tags',
}

# Iterar sobre as chaves do dicionário de mapeamento
for coluna_em_ingles, coluna_em_portugues in nome_coluna_map.items():
    # Verifique se a coluna em inglês existe no DataFrame
    if coluna_em_ingles in df.columns:
        # Renomeie a coluna no DataFrame
        df = df.rename(columns={coluna_em_ingles: coluna_em_portugues})

df

Dados apresentados no relátorio

In [ ]:
# Filtrar as tarefas pela tag
filtro_tag_relatorio = df['Tags'].apply(lambda x: x is not None and 'xxxxxx' in x)

# Filtrar as tarefas pelo mês de conclusão (por exemplo, setembro)
filtro_mes = df['Concluída em'].str.contains("/03/2024")

# Combinar os dois filtros
tarefas_relatorio = df[filtro_tag_relatorio & filtro_mes]

tarefas_relatorio = tarefas_relatorio[["Seção", "Nome da Tarefa",	"Nome da Subtarefa",	"Responsável",	"Notas",	"Concluída em",	"Tags"]]

# Salvar o DataFrame em um arquivo CSV
nome_arquivo_csv = 'xxxxxx.csv'
tarefas_relatorio.to_csv(nome_arquivo_csv, index=False)


tarefas_relatorio


In [ ]:
# Contar o número de tarefas em tarefas_relatorio
numero_de_tarefas = tarefas_relatorio.shape[0]

# Mostrar o resultado
print(f"Número de tarefas: {numero_de_tarefas}")


Número de tarefas: 6
